## UNTAPPED ENERGY DUC DATATHON - DATA CLEANING WELL HEADER

### WORKFLOW
    - Import Well Header
    - Drop unnecessary features and check columns from dataset preparation
    - Create dictionary / reference for Operator to potentially save memory for full dataset
    - Fill NAN's as needed
    - To DateTime for date columns
    - save cleaned file
    - Plot BH locations
    - Check for TVD Outliers / Data Errors
    

In [ ]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [ ]:
# Load files
Wells = pd.read_csv('WellHeader_Datathon.csv')

In [ ]:
cols = Wells.columns
cols

In [ ]:
Wells.head()

In [ ]:
keep_columns = ['EPAssetsId','Province', 'UWI',
       'CurrentOperator', 'CurrentOperatorParent', 'CurrentOperatorParentName',
       'CurrentStatus', 'CurrentStatusStandardised', 'WellType',
       'WellTypeStandardised', 'WellName', 'Formation', 'Field', 'Pool',
       'Surf_Location', 'Surf_Longitude', 'Surf_Latitude','BH_Location',
       'BH_Longitude', 'BH_Latitude','TotalDepth', 'LaheeClass',
       'Confidential','KBElevation', 'SpudDate', 'FinalDrillDate','RigReleaseDate','DaysDrilling', 'TVD', 'WellProfile',
       'StatusDate', 'UnitFlag']
drop_columns = list( set(cols) - set(keep_columns) )


In [ ]:
Wells = Wells.drop(Wells[drop_columns], axis = 1)

In [ ]:
Wells.head()

In [ ]:
Wells.info()

In [ ]:
# Convert to datetime
date_cols = ['SpudDate', 'FinalDrillDate', 'RigReleaseDate', 'StatusDate']
for col in date_cols:
    Wells[col] = pd.to_datetime(Wells[col], infer_datetime_format=True)

In [ ]:
Wells['UnitFlag'].fillna('No', inplace = True)

In [ ]:
sum(Wells['FinalDrillDate'].isnull())

In [ ]:
Wells['FinalDrillDate'].fillna(Wells['RigReleaseDate'] - pd.DateOffset(days=2) ,inplace=True)

In [ ]:
sum(Wells['DaysDrilling']==0), sum(Wells['DaysDrilling'].isnull())

In [ ]:
Zero_DrillDays = pd.Series(Wells['EPAssetsId'][Wells['DaysDrilling']==0])
Zero_DrillDays.shape

In [ ]:
for ID in Zero_DrillDays.index:
    Wells['DaysDrilling'][ID] = (Wells['FinalDrillDate'][ID] - Wells['SpudDate'][ID]).days

In [ ]:
Wells.info()

In [ ]:
Operators = Wells[['CurrentOperatorParent', 'CurrentOperatorParentName']]
Operators.columns = ['Op_Num', 'Op_Name']
Operators.head()

In [ ]:
Operators.drop_duplicates(inplace = True)
Operators.index=range(Operators.shape[0])

In [ ]:
Operators

In [ ]:
Operators.to_csv('Operators_List.csv')

In [ ]:
encoder_Formation = LabelEncoder()
encoder_Formation.fit(Wells['Formation'])
Wells['Formation_C'] = encoder_Formation.transform(Wells['Formation'])

encoder_Field = LabelEncoder()
encoder_Field.fit(Wells['Field'])
Wells['Field_C'] = encoder_Field.transform(Wells['Field'])

encoder_Type = LabelEncoder()
encoder_Type.fit(Wells['WellTypeStandardised'])
Wells['Type_C'] = encoder_Type.transform(Wells['WellTypeStandardised'])


In [ ]:
Formations = Wells[['Formation_C', 'Formation']]
Formations.columns = ['Formation_Num', 'Formation_Name']

Formations.drop_duplicates(inplace = True)
Formations = Formations.sort_values('Formation_Num')
Formations.index=range(Formations.shape[0])
Formations

In [ ]:
Fields = Wells[['Field_C', 'Field']]
Fields.columns = ['Field_Num', 'Field_Name']

Fields.drop_duplicates(inplace = True)
Fields = Fields.sort_values('Field_Num')
Fields.index=range(Fields.shape[0])
Fields

In [ ]:
Formations.to_csv('FormationNames_Nums.csv')

In [ ]:
Fields.to_csv('FieldNames_Nums.csv')

In [ ]:
Wells.head()

In [ ]:
Wells.info()

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize = (8, 11))

ax1.scatter(x=Wells['Surf_Longitude'][Wells['Type_C'] == 0], y=Wells['Surf_Latitude'][Wells['Type_C'] == 0], 
            marker = 'o', label = 'Gas', c = Wells['Field_C'][Wells['Type_C'] == 0])
ax1.scatter(x=Wells['Surf_Longitude'][Wells['Type_C'] == 1], y=Wells['Surf_Latitude'][Wells['Type_C'] == 1], 
            marker = 'D', label = 'Oil', c = Wells['Field_C'][Wells['Type_C'] == 1])

ax1.plot([-105, -125] , [49, 49 ], '--', c = 'r')
ax1.plot([-105, -125], [60, 60], '--', c = 'r')
ax1.plot([-110, -114.03, -120, -120, -110, -110], [49, 49, 53.5, 60, 60, 49 ], '-')

ax1.set_title('Sample Data Surface Well Location by Field and Type')
ax1.set_xlabel('Latutude')
ax1.set_ylabel('Longitude')
ax1.set_xlim(-125, -105)
ax1.set_ylim(48, 62)
ax1.grid()

ax1.legend()
plt.show()

In [ ]:
sum(Wells['Surf_Latitude']==0)

In [ ]:
sum(Wells['BH_Latitude'].isnull())

In [ ]:
Wells[Wells['Surf_Latitude']==0]

### There are 12 BC wells with 0's for Lat & Long.  Fix these with corresponding Lat & Long from other wells with the same surface location.

In [ ]:
Ref_LatLongs = pd.read_csv('BC_Wells_with_0_Lat_Longs.csv')

In [ ]:
Ref_LatLongs

In [ ]:
target = Ref_LatLongs['EPAssetsId']
reference = Ref_LatLongs['Lat_Long_Reference Well EPAssetId']

for sample in range(len(target)):
   # Ref_ID = Ref_LatLongs['Lat_Lonf_Reference'][Ref_LatLongs['EPAssetsId']==ID]

    Wells['Surf_Longitude'][Wells['EPAssetsId']==target[sample]] = Wells['Surf_Longitude'][Wells['EPAssetsId']==reference[sample]]
    Wells['Surf_Latitude'][Wells['EPAssetsId']==target[sample]] = Wells['Surf_Latitude'][Wells['EPAssetsId']==reference[sample]]

#### Check for Outliers on TVD by searching for horizontal or directional wells that have the same TVD as Total Depth, or have identical Surface Lat & Long as BH Lat & Long, or identical Surface & BH Location to account for missing Lat / Long or missing TVD

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize = (8, 8))

plt.scatter(Wells['TotalDepth'], Wells['TVD'], label = 'All Wells with TVD Entry')
plt.scatter(Wells['TotalDepth'][Wells['WellProfile']=='Vertical'], Wells['TVD'][Wells['WellProfile']=='Vertical'], label = 'Vertical Wells')
plt.title('Wells TVD vs Total Depth')
plt.xlabel('Total Depth')
plt.ylabel('TVD')
plt.legend()
plt.grid()
plt.show

In [ ]:
#Plot wells that TVD and Total Depth Match, Overlay Vertical wells

plt.scatter(Wells['TotalDepth'][Wells['TotalDepth'] == Wells['TVD']], Wells['TVD'][Wells['TotalDepth'] == Wells['TVD']], label = 'All Wells with TVD Entry')
plt.scatter(Wells['TotalDepth'][Wells['WellProfile']=='Vertical'], Wells['TVD'][Wells['WellProfile']=='Vertical'], label = 'Vertical Wells')
plt.title('Wells Where TVD is Indentical to Total Depth')
plt.xlabel('Total Depth')
plt.ylabel('TVD')
plt.legend()
plt.grid()
plt.show

In [ ]:
# PLot Wells where TVD is greater than 90% of Total Depth

plt.scatter(Wells['TotalDepth'][Wells['TVD']>0.9* Wells['TotalDepth']], Wells['TVD'][Wells['TVD']>0.9* Wells['TotalDepth']], label = 'All Wells with TVD Entry')
plt.scatter(Wells['TotalDepth'][Wells['WellProfile']=='Vertical'], Wells['TVD'][Wells['WellProfile']=='Vertical'], label = 'Vertical Wells')
plt.title('Wells Where TVD is > 90% of Total Depth')
plt.xlabel('Total Depth')
plt.ylabel('TVD')
plt.legend()
plt.grid()
plt.show

In [ ]:
# Filter for Wells where TVD is greater than 90% of Total Depth and drop Vertical Wells
Possible_Outliers = (Wells[Wells['TVD']>0.9* Wells['TotalDepth']])
Possible_Outliers = Possible_Outliers[['EPAssetsId','Province', 'UWI',
       'Surf_Location', 'Surf_Longitude', 'Surf_Latitude','BH_Location',
       'BH_Longitude', 'BH_Latitude','WellProfile', 'TotalDepth','TVD','KBElevation','WellType',
       'WellTypeStandardised', 'WellName', 'Formation', 'Field', 'Pool']][Possible_Outliers['WellProfile']!='Vertical']

In [ ]:
# Print possible outliers
Possible_Outliers[['EPAssetsId','Province', 'UWI',
       'Surf_Location', 'Surf_Longitude', 'Surf_Latitude','BH_Location',
       'BH_Longitude', 'BH_Latitude','WellProfile', 'TotalDepth','TVD','KBElevation','WellType',
       'WellTypeStandardised', 'WellName', 'Formation', 'Field', 'Pool']]

In [ ]:
# Filter for horizontal or directional well that have identical surface & BH Longitude
Possible_Outliers[Possible_Outliers['Surf_Longitude']==Possible_Outliers['BH_Longitude']]

In [ ]:
subheader = ['EPAssetsId','Surf_Location','BH_Location','Surf_Longitude','Surf_Latitude','BH_Longitude','BH_Latitude','TVD','TotalDepth']

In [ ]:
# Filter for horizontal or directional well that have identical surface & BH Latitude
Possible_Outliers[subheader][Possible_Outliers['Surf_Latitude']==Possible_Outliers['BH_Latitude']]

#### These return the same outlier well

#### Check All Wells for Outliers via identical surface & BH Lat & Long to get wells that do not have a TVD

In [ ]:
# Possible Outliers with identical Longitude, Filter out Vertical Wells

PossOutlier_Long = Wells[Wells['Surf_Longitude']==Wells['BH_Longitude']]
PossOutlier_Long = PossOutlier_Long[['EPAssetsId','Province', 'UWI',
       'Surf_Location', 'Surf_Longitude', 'Surf_Latitude','BH_Location',
       'BH_Longitude', 'BH_Latitude','WellProfile', 'TotalDepth','TVD','KBElevation','WellType',
       'WellTypeStandardised', 'WellName', 'Formation', 'Field', 'Pool']][PossOutlier_Long['WellProfile']!='Vertical']
print('Identical Longitude Filter found ', len(PossOutlier_Long), ' possible outliers')

In [ ]:
PossOutlier_Long

#### Only #2585178 is an outlier from data errors

In [ ]:
# Possible Outliers with identical Longitude, Filter out Vertical Wells

PossOutlier_Lat = Wells[Wells['Surf_Latitude']==Wells['BH_Latitude']]
PossOutlier_Lat = PossOutlier_Lat[['EPAssetsId','Province', 'UWI',
       'Surf_Location', 'Surf_Longitude', 'Surf_Latitude','BH_Location',
       'BH_Longitude', 'BH_Latitude','WellProfile', 'TotalDepth','TVD','KBElevation','WellType',
       'WellTypeStandardised', 'WellName', 'Formation', 'Field', 'Pool']][PossOutlier_Lat['WellProfile']!='Vertical']
print('Identical Longitude Filter found ', len(PossOutlier_Lat), ' possible outliers')

In [ ]:
PossOutlier_Lat

#### Only #2585178 is an outlier from data errors

#### Check All Wells for Outliers via identical surface & BH Location Strings to get wells that do not have a TVD

In [ ]:
# Possible Outliers with identical Longitude, Filter out Vertical Wells

PossOutlier_Loc = Wells[Wells['Surf_Location']==Wells['BH_Location']]
PossOutlier_Loc = PossOutlier_Loc[['EPAssetsId','Province', 'UWI',
       'Surf_Location', 'Surf_Longitude', 'Surf_Latitude','BH_Location',
       'BH_Longitude', 'BH_Latitude','WellProfile', 'TotalDepth','TVD','KBElevation','WellType',
       'WellTypeStandardised', 'WellName', 'Formation', 'Field', 'Pool']][PossOutlier_Loc['WellProfile']!='Vertical']
print('Identical Longitude Filter found ', len(PossOutlier_Loc), ' possible outliers')

In [ ]:
PossOutlier_Loc

#### None of these are data errors, there is only a slight directional component to these wells

### ! Only #2585178 is an outlier from data errors and needs the Lat Long investigated & corrected

In [ ]:
# REPLACE BH LONGITUDE AND TVD FROM INVESTIGATION OF THIS WELL AND ADJACENT WELLS IN csv FILE AND TVD REGRESSION ON SAMPLE DATA

Wells['BH_Longitude'][Wells['EPAssetsId']== 2585178] = -115.1173932
Wells['TVD'][Wells['EPAssetsId']== 2585178] = 1576.6

Wells[Wells['EPAssetsId']== 2585178][['EPAssetsId', 'BH_Longitude', 'TVD']]


In [ ]:
Wells[subheader][Wells['EPAssetsId'] == 1170677]

In [ ]:
Offset_to_1170677_with_TVD = [1170140,1170226,1170677,1170296,1170360]

In [ ]:
Wells[subheader][Wells['EPAssetsId'].isin(Offset_to_1170677_with_TVD)]

In [ ]:
# Correct erroneous TVD for well 1170677 1-9-31-19 W3 from 134m to 740m
Wells['TVD'][Wells['EPAssetsId']== 1170677] = np.mean([741.6, 737.68, 740.09, 735.4])

In [ ]:
Wells.to_csv('WellHeader_Clean.csv')